In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
import cftime
from datetime import date, timedelta

In [15]:
case = "fanv3_1th"
path = "/scratch/jinmuluo/archive/" + case + "/lnd/hist/" 
start_date = date(2010, 1, 1)
end_date = date(2013, 1, 1)
delta = end_date - start_date
fields = ["N2O_DENITRIFY_TOTAL", "N2O_NITRIFY_TOTAL", "NOx_DENITRIFY_TOTAL", "NOx_NITRIFY_TOTAL"]
fanCTSM = []
for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    day = day.strftime('%Y-%m-%d')
    fanCTSM.append(path + case + ".clm2.h6." + day +"-00000.nc")
print(fanCTSM[-1])

/scratch/jinmuluo/archive/fanv3_1th/lnd/hist/fanv3_1th.clm2.h6.2013-01-01-00000.nc


In [ ]:
def preprocess (ds, fields = fields):
    return(ds[fields])

def fix_time(ds):  
    ndays = len(ds.time)
    yr0 = ds['time.year'][0].values
    ds['time'] =xr.cftime_range(str(yr0),periods=ndays,freq='D')    
    return ds

fix_time(xr.open_mfdataset(fanCTSM, decode_times=True, preprocess=preprocess))

In [ ]:
def single_point(ds):
    

In [ ]:
def total(ds):